# OpenAI APIs - Completions

SGLang provides OpenAI-compatible APIs to enable a smooth transition from OpenAI services to self-hosted local models.
A complete reference for the API is available in the [OpenAI API Reference](https://platform.openai.com/docs/api-reference).

This tutorial covers the following popular APIs:

- `chat/completions`
- `completions`
- `batches`

Check out other tutorials to learn about [vision APIs](https://docs.sglang.ai/backend/openai_api_vision.html) for vision-language models and [embedding APIs](https://docs.sglang.ai/backend/openai_api_embeddings.html) for embedding models.

## Launch A Server

Launch the server in your terminal and wait for it to initialize.

In [1]:
from sglang.test.test_utils import is_in_ci

if is_in_ci():
    from patch import launch_server_cmd
else:
    from sglang.utils import launch_server_cmd

from sglang.utils import wait_for_server, print_highlight, terminate_process


server_process, port = launch_server_cmd(
    "python3 -m sglang.launch_server --model-path qwen/qwen2.5-0.5b-instruct --host 0.0.0.0 --mem-fraction-static 0.8"
)

wait_for_server(f"http://localhost:{port}")
print(f"Server started on http://localhost:{port}")

[2025-04-22 07:10:26] server_args=ServerArgs(model_path='qwen/qwen2.5-0.5b-instruct', tokenizer_path='qwen/qwen2.5-0.5b-instruct', tokenizer_mode='auto', skip_tokenizer_init=False, enable_tokenizer_batch_encode=False, load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='qwen/qwen2.5-0.5b-instruct', chat_template=None, completion_template=None, is_embedding=False, revision=None, host='0.0.0.0', port=39029, mem_fraction_static=0.8, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, stream_interval=1, stream_output=False, random_seed=736568025, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None, base_gpu_id=0, gpu_id_step=1, log_level='info', log_level_http=None,

[2025-04-22 07:10:36 TP0] Attention backend not set. Use fa3 backend by default.
[2025-04-22 07:10:36 TP0] Init torch distributed begin.


[2025-04-22 07:10:36 TP0] Init torch distributed ends. mem usage=0.00 GB
[2025-04-22 07:10:36 TP0] Load weight begin. avail mem=60.58 GB
[2025-04-22 07:10:36 TP0] Ignore import error when loading sglang.srt.models.llama4. 


[2025-04-22 07:10:37 TP0] Using model weights format ['*.safetensors']
[2025-04-22 07:10:37 TP0] No model.safetensors.index.json found in remote.
Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  5.32it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  5.31it/s]

[2025-04-22 07:10:37 TP0] Load weight end. type=Qwen2ForCausalLM, dtype=torch.bfloat16, avail mem=59.52 GB, mem usage=1.06 GB.
[2025-04-22 07:10:37 TP0] KV Cache is allocated. #tokens: 20480, K size: 0.12 GB, V size: 0.12 GB
[2025-04-22 07:10:37 TP0] Memory pool end. avail mem=59.11 GB
[2025-04-22 07:10:37 TP0] 

CUDA Graph is DISABLED.
This will cause significant performance degradation.
CUDA Graph should almost never be disabled in most usage scenarios.
If you encounter OOM issues, please try setting --mem-fraction-static to a lower value (such as 0.8 or 0.7) instead of disabling CUDA Graph.



[2025-04-22 07:10:38 TP0] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=32768


[2025-04-22 07:10:38] INFO:     Started server process [1461718]
[2025-04-22 07:10:38] INFO:     Waiting for application startup.
[2025-04-22 07:10:38] INFO:     Application startup complete.
[2025-04-22 07:10:38] INFO:     Uvicorn running on http://0.0.0.0:39029 (Press CTRL+C to quit)


[2025-04-22 07:10:39] INFO:     127.0.0.1:33142 - "GET /v1/models HTTP/1.1" 200 OK


[2025-04-22 07:10:39] INFO:     127.0.0.1:33148 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-04-22 07:10:39 TP0] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


Server started on http://localhost:39029


## Chat Completions

### Usage

The server fully implements the OpenAI API.
It will automatically apply the chat template specified in the Hugging Face tokenizer, if one is available.
You can also specify a custom chat template with `--chat-template` when launching the server.

In [2]:
import openai

client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="qwen/qwen2.5-0.5b-instruct",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-04-22 07:10:44 TP0] Prefill batch. #new-seq: 1, #new-token: 37, #cached-token: 0, token usage: 0.00, #running-req: 1, #queue-req: 0, 


[2025-04-22 07:10:45] INFO:     127.0.0.1:33154 - "POST /generate HTTP/1.1" 200 OK
[2025-04-22 07:10:45] The server is fired up and ready to roll!


[2025-04-22 07:10:45] INFO:     127.0.0.1:60394 - "POST /v1/chat/completions HTTP/1.1" 200 OK


### Parameters

The chat completions API accepts OpenAI Chat Completions API's parameters. Refer to [OpenAI Chat Completions API](https://platform.openai.com/docs/api-reference/chat/create) for more details.

Here is an example of a detailed chat completion request:

In [3]:
response = client.chat.completions.create(
    model="qwen/qwen2.5-0.5b-instruct",
    messages=[
        {
            "role": "system",
            "content": "You are a knowledgeable historian who provides concise responses.",
        },
        {"role": "user", "content": "Tell me about ancient Rome"},
        {
            "role": "assistant",
            "content": "Ancient Rome was a civilization centered in Italy.",
        },
        {"role": "user", "content": "What were their major achievements?"},
    ],
    temperature=0.3,  # Lower temperature for more focused responses
    max_tokens=128,  # Reasonable length for a concise response
    top_p=0.95,  # Slightly higher for better fluency
    presence_penalty=0.2,  # Mild penalty to avoid repetition
    frequency_penalty=0.2,  # Mild penalty for more natural language
    n=1,  # Single response is usually more stable
    seed=42,  # Keep for reproducibility
)

print_highlight(response.choices[0].message.content)

[2025-04-22 07:10:45 TP0] Prefill batch. #new-seq: 1, #new-token: 49, #cached-token: 5, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-04-22 07:10:45 TP0] Decode batch. #running-req: 1, #token: 56, token usage: 0.00, gen throughput (token/s): 6.68, #queue-req: 0, 


[2025-04-22 07:10:45 TP0] Decode batch. #running-req: 1, #token: 96, token usage: 0.00, gen throughput (token/s): 130.96, #queue-req: 0, 


[2025-04-22 07:10:45 TP0] Decode batch. #running-req: 1, #token: 136, token usage: 0.01, gen throughput (token/s): 130.57, #queue-req: 0, 


[2025-04-22 07:10:46 TP0] Decode batch. #running-req: 1, #token: 176, token usage: 0.01, gen throughput (token/s): 133.96, #queue-req: 0, 
[2025-04-22 07:10:46] INFO:     127.0.0.1:60394 - "POST /v1/chat/completions HTTP/1.1" 200 OK


Streaming mode is also supported.

In [4]:
stream = client.chat.completions.create(
    model="qwen/qwen2.5-0.5b-instruct",
    messages=[{"role": "user", "content": "Say this is a test"}],
    stream=True,
)
for chunk in stream:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="")

[2025-04-22 07:10:46] INFO:     127.0.0.1:60394 - "POST /v1/chat/completions HTTP/1.1" 200 OK
[2025-04-22 07:10:46 TP0] Prefill batch. #new-seq: 1, #new-token: 10, #cached-token: 24, token usage: 0.00, #running-req: 0, #queue-req: 0, 
Yes, I can say that this is a test. I'm a large language model created

 by Alibaba Cloud. My purpose is to assist

 you and provide you with accurate[2025-04-22 07:10:46 TP0] Decode batch. #running-req: 1, #token: 68, token usage: 0.00, gen throughput (token/s): 124.97, #queue-req: 0, 
 and up-to-date information. If you have any questions or

 need help with anything, feel free to ask

.

## Completions

### Usage
Completions API is similar to Chat Completions API, but without the `messages` parameter or chat templates.

In [5]:
response = client.completions.create(
    model="qwen/qwen2.5-0.5b-instruct",
    prompt="List 3 countries and their capitals.",
    temperature=0,
    max_tokens=64,
    n=1,
    stop=None,
)

print_highlight(f"Response: {response}")

[2025-04-22 07:10:46 TP0] Prefill batch. #new-seq: 1, #new-token: 8, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-04-22 07:10:46 TP0] Decode batch. #running-req: 1, #token: 26, token usage: 0.00, gen throughput (token/s): 128.27, #queue-req: 0, 


[2025-04-22 07:10:47 TP0] Decode batch. #running-req: 1, #token: 66, token usage: 0.00, gen throughput (token/s): 129.42, #queue-req: 0, 
[2025-04-22 07:10:47] INFO:     127.0.0.1:60394 - "POST /v1/completions HTTP/1.1" 200 OK


### Parameters

The completions API accepts OpenAI Completions API's parameters.  Refer to [OpenAI Completions API](https://platform.openai.com/docs/api-reference/completions/create) for more details.

Here is an example of a detailed completions request:

In [6]:
response = client.completions.create(
    model="qwen/qwen2.5-0.5b-instruct",
    prompt="Write a short story about a space explorer.",
    temperature=0.7,  # Moderate temperature for creative writing
    max_tokens=150,  # Longer response for a story
    top_p=0.9,  # Balanced diversity in word choice
    stop=["\n\n", "THE END"],  # Multiple stop sequences
    presence_penalty=0.3,  # Encourage novel elements
    frequency_penalty=0.3,  # Reduce repetitive phrases
    n=1,  # Generate one completion
    seed=123,  # For reproducible results
)

print_highlight(f"Response: {response}")

[2025-04-22 07:10:47 TP0] Prefill batch. #new-seq: 1, #new-token: 9, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-04-22 07:10:47 TP0] Decode batch. #running-req: 1, #token: 43, token usage: 0.00, gen throughput (token/s): 124.61, #queue-req: 0, 


[2025-04-22 07:10:47 TP0] Decode batch. #running-req: 1, #token: 83, token usage: 0.00, gen throughput (token/s): 120.03, #queue-req: 0, 
[2025-04-22 07:10:47] INFO:     127.0.0.1:60394 - "POST /v1/completions HTTP/1.1" 200 OK


## Structured Outputs (JSON, Regex, EBNF)

For OpenAI compatible structed outputs API, refer to [Structured Outputs](https://docs.sglang.ai/backend/structured_outputs.html#OpenAI-Compatible-API) for more details.


## Batches

Batches API for chat completions and completions are also supported. You can upload your requests in `jsonl` files, create a batch job, and retrieve the results when the batch job is completed (which takes longer but costs less).

The batches APIs are:

- `batches`
- `batches/{batch_id}/cancel`
- `batches/{batch_id}`

Here is an example of a batch job for chat completions, completions are similar.


In [7]:
import json
import time
from openai import OpenAI

client = OpenAI(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

requests = [
    {
        "custom_id": "request-1",
        "method": "POST",
        "url": "/chat/completions",
        "body": {
            "model": "qwen/qwen2.5-0.5b-instruct",
            "messages": [
                {"role": "user", "content": "Tell me a joke about programming"}
            ],
            "max_tokens": 50,
        },
    },
    {
        "custom_id": "request-2",
        "method": "POST",
        "url": "/chat/completions",
        "body": {
            "model": "qwen/qwen2.5-0.5b-instruct",
            "messages": [{"role": "user", "content": "What is Python?"}],
            "max_tokens": 50,
        },
    },
]

input_file_path = "batch_requests.jsonl"

with open(input_file_path, "w") as f:
    for req in requests:
        f.write(json.dumps(req) + "\n")

with open(input_file_path, "rb") as f:
    file_response = client.files.create(file=f, purpose="batch")

batch_response = client.batches.create(
    input_file_id=file_response.id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
)

print_highlight(f"Batch job created with ID: {batch_response.id}")

[2025-04-22 07:10:47] INFO:     127.0.0.1:60410 - "POST /v1/files HTTP/1.1" 200 OK
[2025-04-22 07:10:47] INFO:     127.0.0.1:60410 - "POST /v1/batches HTTP/1.1" 200 OK


[2025-04-22 07:10:47 TP0] Prefill batch. #new-seq: 2, #new-token: 20, #cached-token: 48, token usage: 0.00, #running-req: 0, #queue-req: 0, 


In [8]:
while batch_response.status not in ["completed", "failed", "cancelled"]:
    time.sleep(3)
    print(f"Batch job status: {batch_response.status}...trying again in 3 seconds...")
    batch_response = client.batches.retrieve(batch_response.id)

if batch_response.status == "completed":
    print("Batch job completed successfully!")
    print(f"Request counts: {batch_response.request_counts}")

    result_file_id = batch_response.output_file_id
    file_response = client.files.content(result_file_id)
    result_content = file_response.read().decode("utf-8")

    results = [
        json.loads(line) for line in result_content.split("\n") if line.strip() != ""
    ]

    for result in results:
        print_highlight(f"Request {result['custom_id']}:")
        print_highlight(f"Response: {result['response']}")

    print_highlight("Cleaning up files...")
    # Only delete the result file ID since file_response is just content
    client.files.delete(result_file_id)
else:
    print_highlight(f"Batch job failed with status: {batch_response.status}")
    if hasattr(batch_response, "errors"):
        print_highlight(f"Errors: {batch_response.errors}")

[2025-04-22 07:10:48 TP0] Decode batch. #running-req: 2, #token: 118, token usage: 0.01, gen throughput (token/s): 180.61, #queue-req: 0, 


Batch job status: validating...trying again in 3 seconds...
[2025-04-22 07:10:50] INFO:     127.0.0.1:60410 - "GET /v1/batches/batch_ae2232ac-5183-4cd8-bf8c-1e79e5a57a6f HTTP/1.1" 200 OK
Batch job completed successfully!
Request counts: BatchRequestCounts(completed=2, failed=0, total=2)
[2025-04-22 07:10:50] INFO:     127.0.0.1:60410 - "GET /v1/files/backend_result_file-4b07e817-39b1-4f1c-81fb-cb740bdf4577/content HTTP/1.1" 200 OK


[2025-04-22 07:10:50] INFO:     127.0.0.1:60410 - "DELETE /v1/files/backend_result_file-4b07e817-39b1-4f1c-81fb-cb740bdf4577 HTTP/1.1" 200 OK


It takes a while to complete the batch job. You can use these two APIs to retrieve the batch job status or cancel the batch job.

1. `batches/{batch_id}`: Retrieve the batch job status.
2. `batches/{batch_id}/cancel`: Cancel the batch job.

Here is an example to check the batch job status.

In [9]:
import json
import time
from openai import OpenAI

client = OpenAI(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

requests = []
for i in range(20):
    requests.append(
        {
            "custom_id": f"request-{i}",
            "method": "POST",
            "url": "/chat/completions",
            "body": {
                "model": "qwen/qwen2.5-0.5b-instruct",
                "messages": [
                    {
                        "role": "system",
                        "content": f"{i}: You are a helpful AI assistant",
                    },
                    {
                        "role": "user",
                        "content": "Write a detailed story about topic. Make it very long.",
                    },
                ],
                "max_tokens": 64,
            },
        }
    )

input_file_path = "batch_requests.jsonl"
with open(input_file_path, "w") as f:
    for req in requests:
        f.write(json.dumps(req) + "\n")

with open(input_file_path, "rb") as f:
    uploaded_file = client.files.create(file=f, purpose="batch")

batch_job = client.batches.create(
    input_file_id=uploaded_file.id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
)

print_highlight(f"Created batch job with ID: {batch_job.id}")
print_highlight(f"Initial status: {batch_job.status}")

time.sleep(10)

max_checks = 5
for i in range(max_checks):
    batch_details = client.batches.retrieve(batch_id=batch_job.id)

    print_highlight(
        f"Batch job details (check {i+1} / {max_checks}) // ID: {batch_details.id} // Status: {batch_details.status} // Created at: {batch_details.created_at} // Input file ID: {batch_details.input_file_id} // Output file ID: {batch_details.output_file_id}"
    )
    print_highlight(
        f"<strong>Request counts: Total: {batch_details.request_counts.total} // Completed: {batch_details.request_counts.completed} // Failed: {batch_details.request_counts.failed}</strong>"
    )

    time.sleep(3)

[2025-04-22 07:10:51] INFO:     127.0.0.1:60414 - "POST /v1/files HTTP/1.1" 200 OK
[2025-04-22 07:10:51] INFO:     127.0.0.1:60414 - "POST /v1/batches HTTP/1.1" 200 OK


[2025-04-22 07:10:51 TP0] Prefill batch. #new-seq: 18, #new-token: 548, #cached-token: 54, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-04-22 07:10:51 TP0] Prefill batch. #new-seq: 2, #new-token: 62, #cached-token: 6, token usage: 0.03, #running-req: 18, #queue-req: 0, 


[2025-04-22 07:10:51 TP0] Decode batch. #running-req: 20, #token: 1143, token usage: 0.06, gen throughput (token/s): 179.63, #queue-req: 0, 


[2025-04-22 07:11:01] INFO:     127.0.0.1:47622 - "GET /v1/batches/batch_efb34a1f-b858-44c9-bcfc-3f196ebd5ad6 HTTP/1.1" 200 OK


[2025-04-22 07:11:04] INFO:     127.0.0.1:47622 - "GET /v1/batches/batch_efb34a1f-b858-44c9-bcfc-3f196ebd5ad6 HTTP/1.1" 200 OK


[2025-04-22 07:11:07] INFO:     127.0.0.1:47622 - "GET /v1/batches/batch_efb34a1f-b858-44c9-bcfc-3f196ebd5ad6 HTTP/1.1" 200 OK


[2025-04-22 07:11:10] INFO:     127.0.0.1:47622 - "GET /v1/batches/batch_efb34a1f-b858-44c9-bcfc-3f196ebd5ad6 HTTP/1.1" 200 OK


[2025-04-22 07:11:13] INFO:     127.0.0.1:47622 - "GET /v1/batches/batch_efb34a1f-b858-44c9-bcfc-3f196ebd5ad6 HTTP/1.1" 200 OK


Here is an example to cancel a batch job.

In [10]:
import json
import time
from openai import OpenAI
import os

client = OpenAI(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

requests = []
for i in range(5000):
    requests.append(
        {
            "custom_id": f"request-{i}",
            "method": "POST",
            "url": "/chat/completions",
            "body": {
                "model": "qwen/qwen2.5-0.5b-instruct",
                "messages": [
                    {
                        "role": "system",
                        "content": f"{i}: You are a helpful AI assistant",
                    },
                    {
                        "role": "user",
                        "content": "Write a detailed story about topic. Make it very long.",
                    },
                ],
                "max_tokens": 128,
            },
        }
    )

input_file_path = "batch_requests.jsonl"
with open(input_file_path, "w") as f:
    for req in requests:
        f.write(json.dumps(req) + "\n")

with open(input_file_path, "rb") as f:
    uploaded_file = client.files.create(file=f, purpose="batch")

batch_job = client.batches.create(
    input_file_id=uploaded_file.id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
)

print_highlight(f"Created batch job with ID: {batch_job.id}")
print_highlight(f"Initial status: {batch_job.status}")

time.sleep(10)

try:
    cancelled_job = client.batches.cancel(batch_id=batch_job.id)
    print_highlight(f"Cancellation initiated. Status: {cancelled_job.status}")
    assert cancelled_job.status == "cancelling"

    # Monitor the cancellation process
    while cancelled_job.status not in ["failed", "cancelled"]:
        time.sleep(3)
        cancelled_job = client.batches.retrieve(batch_job.id)
        print_highlight(f"Current status: {cancelled_job.status}")

    # Verify final status
    assert cancelled_job.status == "cancelled"
    print_highlight("Batch job successfully cancelled")

except Exception as e:
    print_highlight(f"Error during cancellation: {e}")
    raise e

finally:
    try:
        del_response = client.files.delete(uploaded_file.id)
        if del_response.deleted:
            print_highlight("Successfully cleaned up input file")
        if os.path.exists(input_file_path):
            os.remove(input_file_path)
            print_highlight("Successfully deleted local batch_requests.jsonl file")
    except Exception as e:
        print_highlight(f"Error cleaning up: {e}")
        raise e

[2025-04-22 07:11:16] INFO:     127.0.0.1:56616 - "POST /v1/files HTTP/1.1" 200 OK
[2025-04-22 07:11:16] INFO:     127.0.0.1:56616 - "POST /v1/batches HTTP/1.1" 200 OK


[2025-04-22 07:11:17 TP0] Prefill batch. #new-seq: 34, #new-token: 440, #cached-token: 706, token usage: 0.03, #running-req: 0, #queue-req: 0, 
[2025-04-22 07:11:17 TP0] Prefill batch. #new-seq: 103, #new-token: 3090, #cached-token: 449, token usage: 0.05, #running-req: 34, #queue-req: 625, 


[2025-04-22 07:11:17 TP0] Decode batch. #running-req: 137, #token: 4524, token usage: 0.22, gen throughput (token/s): 38.40, #queue-req: 4863, 
[2025-04-22 07:11:17 TP0] Prefill batch. #new-seq: 24, #new-token: 720, #cached-token: 120, token usage: 0.29, #running-req: 136, #queue-req: 4839, 
[2025-04-22 07:11:17 TP0] Prefill batch. #new-seq: 7, #new-token: 210, #cached-token: 35, token usage: 0.33, #running-req: 159, #queue-req: 4832, 


[2025-04-22 07:11:17 TP0] Decode batch. #running-req: 163, #token: 11391, token usage: 0.56, gen throughput (token/s): 15577.71, #queue-req: 4832, 


[2025-04-22 07:11:18 TP0] Decode batch. #running-req: 161, #token: 17757, token usage: 0.87, gen throughput (token/s): 17576.65, #queue-req: 4832, 
[2025-04-22 07:11:18 TP0] Decode out of memory happened. #retracted_reqs: 23, #new_token_ratio: 0.6007 -> 0.9233


...

[2025-04-22 07:11:25 TP0] Prefill batch. #new-seq: 2, #new-token: 60, #cached-token: 12, token usage: 0.54, #running-req: 140, #queue-req: 4047, 
[2025-04-22 07:11:25 TP0] Prefill batch. #new-seq: 1, #new-token: 31, #cached-token: 5, token usage: 0.58, #running-req: 141, #queue-req: 4046, 


[2025-04-22 07:11:26 TP0] Decode batch. #running-req: 142, #token: 13977, token usage: 0.68, gen throughput (token/s): 12643.12, #queue-req: 4046, 
[2025-04-22 07:11:26] INFO:     127.0.0.1:42452 - "POST /v1/batches/batch_e5f202a8-2fc1-4123-bb41-c3bb3ffc87a4/cancel HTTP/1.1" 200 OK


[2025-04-22 07:11:26 TP0] Prefill batch. #new-seq: 58, #new-token: 7553, #cached-token: 214, token usage: 0.00, #running-req: 24, #queue-req: 0, 
[2025-04-22 07:11:26 TP0] Decode batch. #running-req: 38, #token: 5334, token usage: 0.26, gen throughput (token/s): 7214.28, #queue-req: 0, 


[2025-04-22 07:11:29] INFO:     127.0.0.1:42452 - "GET /v1/batches/batch_e5f202a8-2fc1-4123-bb41-c3bb3ffc87a4 HTTP/1.1" 200 OK


[2025-04-22 07:11:29] INFO:     127.0.0.1:42452 - "DELETE /v1/files/backend_input_file-a5f43f40-7fb8-49ac-9981-572d5ee735dd HTTP/1.1" 200 OK


In [11]:
terminate_process(server_process)

[2025-04-22 07:11:29] Child process unexpectedly failed with an exit code 9. pid=1462615
[2025-04-22 07:11:29] Child process unexpectedly failed with an exit code 9. pid=1462533
